# Final project

In [1]:
#r "nuget: FSharp.Data, 5.*"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET, 3.*"
#r "nuget: Plotly.NET.Interactive, 3.*"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"
#r "nuget: NovaSBE.Finance, 0.2.0-beta1"
#r "nuget: Deedle"
#r "nuget: Quotes.YahooFinance, 0.0.5"
#r "nuget: DiffSharp-lite"

open System
open FSharp.Data
open FSharp.Stats
open Plotly.NET
open MathNet.Numerics
open MathNet.Numerics.Statistics
open NovaSBE.Finance
open MathNet.Numerics.LinearAlgebra
open NovaSBE.Finance.Ols
open NovaSBE.Finance.Portfolio
open NovaSBE.Finance.French
open Deedle
open Quotes.YahooFinance
open DiffSharp


Installed Packages Deedle, 3.0.0 DiffSharp-lite, 1.0.7 FSharp.Data, 5.0.2 FSharp.Stats, 0.4.11 MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0 NovaSBE.Finance, 0.2.0-beta1 Plotly.NET, 3.0.1 Plotly.NET.Interactive, 3.0.2 Quotes.YahooFinance, 0.0.5

Loading extensions from `/Users/rodrigosimoes/.nuget/packages/plotly.net.interactive/3.0.2/interactive-extensions/dotnet/Plotly.NET.Interactive.dll`

### Import data

In [2]:
let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

let [<Literal>] IdAndReturnsFilePath = "id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "cowc_gr1a.csv"

In [3]:
type IdAndReturnsType = 
    CsvProvider<Sample=IdAndReturnsFilePath,
                ResolutionFolder=ResolutionFolder>

type MySignalType = 
    CsvProvider<MySignalFilePath,
                ResolutionFolder=ResolutionFolder>


In [4]:
let idAndReturnsCsv = IdAndReturnsType.GetSample()
let mySignalCsv = MySignalType.GetSample()

let idAndReturnsRows = idAndReturnsCsv.Rows |> Seq.toList
let mySignalRows = mySignalCsv.Rows |> Seq.toList

In [5]:
let mySignals =
    mySignalRows
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal ->
            let signalRecord: Signal =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  // Signal has negative direction
                  Signal = -signal }
            Some signalRecord)

// look at a few signals
mySignals[..3]

index value 0 { SecurityId = Other "comp_001034_01"\n FormationDate = 01/12/2008 00:00:00\n Signal = 0.042908689 } SecurityId Other "comp_001034_01" Item comp_001034_01 FormationDate 2008-12-01 00:00:00Z Signal 0.042908689 1 { SecurityId = Other "comp_001043_01"\n FormationDate = 01/01/2000 00:00:00\n Signal = -0.0498933902 } SecurityId Other "comp_001043_01" Item comp_001043_01 FormationDate 2000-01-01 00:00:00Z Signal -0.0498933902 2 { SecurityId = Other "comp_001081_01"\n FormationDate = 01/10/2007 00:00:00\n Signal = -0.0042606852 } SecurityId Other "comp_001081_01" Item comp_001081_01 FormationDate 2007-10-01 00:00:00Z Signal -0.0042606852 3 { SecurityId = Other "comp_001095_01"\n FormationDate = 01/06/2002 00:00:00\n Signal = -0.0167255795 } SecurityId Other "comp_001095_01" Item comp_001095_01 FormationDate 2002-06-01 00:00:00Z Signal -0.0167255795

In [6]:
let myReturns =
    idAndReturnsRows
    |> List.choose (fun row -> 
        match row.Ret with
        | None -> None
        | Some ret ->
            let ret: SecurityReturn =
                { SecurityId = Other row.Id
                  Date = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Return= ret }
            Some ret)

// look at a few returns
myReturns[..3]


index value 0 { SecurityId = Other "crsp_86432"\n Date = 01/01/2000 00:00:00\n Return = -0.003906 } SecurityId Other "crsp_86432" Item crsp_86432 Date 2000-01-01 00:00:00Z Return -0.003906 1 { SecurityId = Other "crsp_85640"\n Date = 01/01/2000 00:00:00\n Return = -0.157143 } SecurityId Other "crsp_85640" Item crsp_85640 Date 2000-01-01 00:00:00Z Return -0.157143 2 { SecurityId = Other "crsp_86430"\n Date = 01/01/2000 00:00:00\n Return = 0.285714 } SecurityId Other "crsp_86430" Item crsp_86430 Date 2000-01-01 00:00:00Z Return 0.285714 3 { SecurityId = Other "crsp_85756"\n Date = 01/01/2000 00:00:00\n Return = 0.190685 } SecurityId Other "crsp_85756" Item crsp_85756 Date 2000-01-01 00:00:00Z Return 0.190685

In [7]:
let myMktCaps =
    idAndReturnsRows
    |> List.choose (fun row -> 
        match row.MarketEquity with
        | None -> None
        | Some mktCap ->
            let mktCap: WeightVariable =
                { SecurityId = Other row.Id
                  FormationDate = DateTime(row.Eom.Year, row.Eom.Month, 1)
                  Value = mktCap }
            Some mktCap)

// look at a few market caps
myMktCaps[..3]

index value 0 { SecurityId = Other "crsp_86432"\n FormationDate = 01/01/2000 00:00:00\n Value = 83.161875 } SecurityId Other "crsp_86432" Item crsp_86432 FormationDate 2000-01-01 00:00:00Z Value 83.161875 1 { SecurityId = Other "crsp_85640"\n FormationDate = 01/01/2000 00:00:00\n Value = 377.954 } SecurityId Other "crsp_85640" Item crsp_85640 FormationDate 2000-01-01 00:00:00Z Value 377.954 2 { SecurityId = Other "crsp_86430"\n FormationDate = 01/01/2000 00:00:00\n Value = 42.38325 } SecurityId Other "crsp_86430" Item crsp_86430 FormationDate 2000-01-01 00:00:00Z Value 42.38325 3 { SecurityId = Other "crsp_85756"\n FormationDate = 01/01/2000 00:00:00\n Value = 16.2155 } SecurityId Other "crsp_85756" Item crsp_85756 FormationDate 2000-01-01 00:00:00Z Value 16.2155

### Value-weight portfolios

In [8]:
let strategyName = "Change in COWC"
let backtest = Backtest(returns=myReturns, signals=mySignals, nPortfolios=3, name = strategyName)
let vw = backtest.strategyValueWeighted(myMktCaps)

In [9]:
// Some portfolios and their position
vw.Portfolios[..3]

index value 0 { Name = "Change in COWC"\n Index = 1\n FormationMonth = 01/12/2008 00:00:00\n Positions =\n [{ SecurityId = Other "crsp_91070"\n Weight = 1.393719523e-06 }; { SecurityId = Other "crsp_86251"\n Weight = 1.002456765e-06 };\n { SecurityId = Other "crsp_9... Name Change in COWC Index 1 FormationMonth 2008-12-01 00:00:00Z Positions index value 0 { SecurityId = Other "crsp_91070"\n Weight = 1.393719523e-06 } SecurityId Other "crsp_91070" Item crsp_91070 Weight 1.3937195227322193E-06 1 { SecurityId = Other "crsp_86251"\n Weight = 1.002456765e-06 } SecurityId Other "crsp_86251" Item crsp_86251 Weight 1.0024567652869495E-06 2 { SecurityId = Other "crsp_92243"\n Weight = 7.985680388e-05 } SecurityId Other "crsp_92243" Item crsp_92243 Weight 7.985680387796288E-05 3 { SecurityId = Other "crsp_91512"\n Weight = 5.133257197e-07 } SecurityId Other "crsp_91512" Item crsp_91512 Weight 5.13325719746382E-07 4 { SecurityId = Other "crsp_82530"\n Weight = 3.743947556e-06 } SecurityId Other "crsp_82530" Item crsp_82530 Weight 3.7439475564690517E-06 5 { SecurityId = Other "crsp_79066"\n Weight = 3.25993626e-06 } SecurityId Other "crsp_79066" Item crsp_79066 Weight 3.2599362602864603E-06 6 { SecurityId = Other "crsp_89445"\n Weight = 2.256595912e-05 } SecurityId Other "crsp_89445" Item crsp_89445 Weight 2.2565959123103108E-05 7 { SecurityId = Other "crsp_53373"\n Weight = 1.724564693e-05 } SecurityId Other "crsp_53373" Item crsp_53373 Weight 1.72456469293296E-05 8 { SecurityId = Other "crsp_86887"\n Weight = 9.553593397e-07 } SecurityId Other "crsp_86887" Item crsp_86887 Weight 9.553593397379023E-07 9 { SecurityId = Other "crsp_90752"\n Weight = 9.368789152e-06 } SecurityId Other "crsp_90752" Item crsp_90752 Weight 9.368789152185624E-06 10 { SecurityId = Other "crsp_90855"\n Weight = 5.943823816e-06 } SecurityId Other "crsp_90855" Item crsp_90855 Weight 5.9438238155101275E-06 11 { SecurityId = Other "crsp_86595"\n Weight = 1.147209611e-05 } SecurityId Other "crsp_86595" Item crsp_86595 Weight 1.1472096107398608E-05 12 { SecurityId = Other "crsp_82552"\n Weight = 1.570507153e-05 } SecurityId Other "crsp_82552" Item crsp_82552 Weight 1.5705071532975245E-05 13 { SecurityId = Other "crsp_91078"\n Weight = 2.13466859e-05 } SecurityId Other "crsp_91078" Item crsp_91078 Weight 2.134668590075088E-05 14 { SecurityId = Other "comp_114698_01"\n Weight = 3.281111208e-06 } SecurityId Other "comp_114698_01" Item comp_114698_01 Weight 3.281111207885825E-06 15 { SecurityId = Other "crsp_92088"\n Weight = 4.265016454e-05 } SecurityId Other "crsp_92088" Item crsp_92088 Weight 4.2650164539863356E-05 16 { SecurityId = Other "crsp_81119"\n Weight = 2.834427478e-07 } SecurityId Other "crsp_81119" Item crsp_81119 Weight 2.834427477935354E-07 17 { SecurityId = Other "crsp_89574"\n Weight = 0.0002099150928 } SecurityId Other "crsp_89574" Item crsp_89574 Weight 0.00020991509277028996 18 { SecurityId = Other "crsp_84420"\n Weight = 1.746597589e-05 } SecurityId Other "crsp_84420" Item crsp_84420 Weight 1.7465975891381783E-05 19 { SecurityId = Other "crsp_90169"\n Weight = 7.857196074e-06 } SecurityId Other "crsp_90169" Item crsp_90169 Weight 7.8571960742664E-06 ... (more) 1 { Name = "Change in COWC"\n Index = 2\n FormationMonth = 01/12/2008 00:00:00\n Positions =\n [{ SecurityId = Other "crsp_89920"\n Weight = 1.205195795e-05 }; { SecurityId = Other "crsp_79133"\n Weight = 0.000488902511 };\n { SecurityId = Other "crsp_84... Name Change in COWC Index 2 FormationMonth 2008-12-01 00:00:00Z Positions index value 0 { SecurityId = Other "crsp_89920"\n Weight = 1.205195795e-05 } SecurityId Other "crsp_89920" Item crsp_89920 Weight 1.2051957949406596E-05 1 { SecurityId = Other "crsp_79133"\n Weight = 0.000488902511 } SecurityId Other "crsp_79133" Item crsp_79133 Weight 0.0004889025109676507 2 { SecurityId = Other "crsp_84769"\n Weight = 0.0008662244481 } SecurityId Other "crsp_84769" Item crsp_84769 Weight 0.0008662244480838072 3 { SecurityId = Other "crsp_86949"\n Weig

In [10]:
// Some portfolio returns
vw.Returns[..3]

index value 0 { Name = "Change in COWC"\n Index = 1\n Month = 01/01/2009 00:00:00\n Return = -0.07062805145 } Name Change in COWC Index 1 Month 2009-01-01 00:00:00Z Return -0.07062805145274711 1 { Name = "Change in COWC"\n Index = 2\n Month = 01/01/2009 00:00:00\n Return = -0.06009513391 } Name Change in COWC Index 2 Month 2009-01-01 00:00:00Z Return -0.0600951339117302 2 { Name = "Change in COWC"\n Index = 3\n Month = 01/01/2009 00:00:00\n Return = -0.06354118521 } Name Change in COWC Index 3 Month 2009-01-01 00:00:00Z Return -0.06354118520501853 3 { Name = "Change in COWC"\n Index = 1\n Month = 01/02/2000 00:00:00\n Return = 0.07837517275 } Name Change in COWC Index 1 Month 2000-02-01 00:00:00Z Return 0.07837517274691844

In [11]:
// get  Fama-French 3-Factor asset pricing model data
let ff3Lookup = 
    French.getFF3 French.Frequency.Monthly
    |> Array.map (fun x -> DateTime(x.Date.Year, x.Date.Month, 1), x)
    |> Map

Create porfolios:

- Long-only strategy portfolio is top tercile portfolio

- Long-short strategy portfolio is long the top portfolio and short the bottom portfolio

In [12]:
type SignalPortfolioObs = 
    { Month: DateTime
      Name: string
      ExcessReturn: float }
      
let long =
    vw.Returns
    |> List.filter (fun row -> row.Index = 3)
    |> List.map (fun row ->
        let retx = row.Return - ff3Lookup[row.Month].Rf 
        { Month = row.Month
          Name = "Long"
          ExcessReturn = retx })

let short =
    vw.Returns
    |> List.filter (fun row -> row.Index = 1)
    |> List.map (fun row -> 
        let retx = row.Return - ff3Lookup[row.Month].Rf 
        { Month = row.Month
          Name = "Short"
          ExcessReturn = retx })

let longShort =
    vw.Returns
    |> List.groupBy (fun x -> x.Month)
    |> List.map (fun (month, xs) ->
        let long = xs |> List.find (fun x -> x.Index = 3)
        let short = xs |> List.find (fun x -> x.Index = 1)
        { Month = long.Month
          Name = "Long-short"
          ExcessReturn = long.Return - short.Return })

Functions to plot cumulative graphs

In [13]:
let cumulativeReturn xs =
    let sorted = xs |> List.sortBy (fun (dt, r) -> dt)
    let mutable cr = 1.0
    [ for (dt, r) in sorted do 
        cr <- cr * (1.0 + r)
        (dt, cr - 1.0) ]
let makeCumulativeChart (returns:List<PortfolioReturn>) =
    let firstObs = returns[0]
    returns
    |> List.map (fun x -> x.Month, x.Return)
    |> cumulativeReturn
    |> Chart.Line
    |> Chart.withTraceInfo(Name = $"{firstObs.Name}: {firstObs.Index}")

Graph showing cumulative returns for Long and Long-short portfolios and excess returns of the value-weighted stock market portfolio (MktRf)

In [14]:
let vwMktRf =
    let portfolioMonths = 
        List.concat [long; short; longShort]
        |> List.map(fun x -> x.Month)
    let minMonth = portfolioMonths |> List.min
    let maxMonth = portfolioMonths |> List.max

    ff3Lookup
    |> Map.toSeq
    |> Seq.filter (fun (month, _) -> month >= minMonth && month <= maxMonth)
    |> Seq.map (fun (month, ffData) ->
        { Month = month
          Name = "MktRf"
          ExcessReturn = ffData.MktRf })
    |> Seq.toList

let combinedChart =
    List.concat [long; longShort; vwMktRf]
    |> List.groupBy(fun x -> x.Name)
    |> List.map(fun (name, xs) ->
        xs
        |> List.map (fun x -> x.Month, x.ExcessReturn)
        |> cumulativeReturn
        |> Chart.Line
        |> Chart.withTraceInfo(Name = name))
    |> Chart.combine
    
combinedChart


<!-- Plotly chart will be drawn inside this DIV -->

Graph showing cumulative returns for Long and Long-short portfolios and excess returns of the value-weighted stock market portfolio (MktRf) with a constant leverage applied to each portfolio so that they all have an annualized volatility of 10%

In [15]:
let annualizedVolatility = 0.10

let leverage xs =
    let returns = xs |> List.map (fun (_, r) -> r)
    let stdev = Statistics.StandardDeviation(returns : float list)
    let leverage = annualizedVolatility / stdev
    xs |> List.map (fun (dt, r) -> dt, r * leverage)

let combinedChartLeveraged =
    List.concat [long; longShort; vwMktRf]
    |> List.groupBy(fun x -> x.Name)
    |> List.map(fun (name, xs) ->
        xs
        |> List.map (fun x -> x.Month, x.ExcessReturn)
        |> leverage
        |> cumulativeReturn
        |> Chart.Line
        |> Chart.withTraceInfo(Name = name))
    |> Chart.combine
    
combinedChartLeveraged

<!-- Plotly chart will be drawn inside this DIV -->

### Performance measures

Functions needed

In [16]:
// Annualize excess returns
let annualizeavgExcessReturn (excessReturns: SignalPortfolioObs list)  =
    let excessReturnsSeq = excessReturns |> List.toSeq
    let avgExcessReturn = excessReturnsSeq |> Seq.averageBy (fun x -> x.ExcessReturn)
    avgExcessReturn

// Annualize Sharpe rations
let annualizeSharpeRatio (excessReturns: SignalPortfolioObs list) =
    let excessReturnsSeq = excessReturns |> List.toSeq
    let avgExcessReturn = excessReturnsSeq |> Seq.averageBy (fun x -> x.ExcessReturn)
    let stDevExcessReturn = excessReturnsSeq |> Seq.map (fun x -> x.ExcessReturn) |> Seq.stDev
    let sharpeRatio = avgExcessReturn / stDevExcessReturn
    sharpeRatio * sqrt 12.0

// Filter returns by time period (first half, second half, or full period)
let filterReturnsByTimePeriodsSign (period: string) (returns: SignalPortfolioObs list) =
    let sortedReturns = List.sortBy (fun x -> x.Month) returns
    let midPoint = sortedReturns.Length / 2

    match period with
    | "First Half" -> sortedReturns |> List.take midPoint
    | "Second Half" -> sortedReturns |> List.skip midPoint
    | "Full Period" -> sortedReturns
    | _ -> failwithf "Unexpected time period: %s" period

Code needed to process models

In [17]:
type RegData =
    { Month: DateTime
      ExcessReturn: float 
      MktRf: float
      Hml: float
      Smb: float }

// Filter returns by time period (first half, second half, or full period)
let filterReturnsByTimePeriodsRd (period: string) (returns: RegData list) =
    let sortedReturns = List.sortBy (fun x -> x.Month) returns
    let midPoint = sortedReturns.Length / 2

    match period with
    | "First Half" -> sortedReturns |> List.take midPoint
    | "Second Half" -> sortedReturns |> List.skip midPoint
    | "Full Period" -> sortedReturns
    | _ -> failwithf "Unexpected time period: %s" period

let longShortRd =
    longShort
    |> List.map (fun ls ->
        let ffFactors = ff3Lookup[ls.Month]
        { Month = ls.Month
          ExcessReturn = ls.ExcessReturn
          MktRf = ffFactors.MktRf
          Hml = ffFactors.Hml
          Smb = ffFactors.Smb })
          
let longRd =
    long
    |> List.map (fun ls ->
        let ffFactors = ff3Lookup[ls.Month]
        { Month = ls.Month
          ExcessReturn = ls.ExcessReturn
          MktRf = ffFactors.MktRf
          Hml = ffFactors.Hml
          Smb = ffFactors.Smb })

type RegressionResults =
    { Betas: float array
      Residuals: float array
      Mse: float }

let fitCapmModel (data: RegData list) =
    let capmInputData = 
     data
    |> List.map (fun rd -> [|1.0; rd.MktRf|], rd.ExcessReturn)

    let capmX = capmInputData |> List.map fst |> Array.ofList |> DenseMatrix.ofRowArrays
    let capmY = capmInputData |> List.map snd |> Array.ofList |> DenseVector.ofArray

    let capmBetas = capmX.QR().Solve(capmY)
    let capmResiduals = capmY - (capmX * capmBetas)
    let capmMse = capmResiduals.PointwisePower(2).Mean()
    { Betas = capmBetas.ToArray(); Residuals = capmResiduals.ToArray(); Mse = capmMse }

let fitff3Model (data: RegData list) =
    let ff3InputData = 
        data
        |> List.map (fun rd -> [|1.0; rd.MktRf; rd.Hml; rd.Smb|], rd.ExcessReturn)


    let ff3X = ff3InputData |> List.map fst |> Array.ofList |> DenseMatrix.ofRowArrays
    let ff3Y = ff3InputData |> List.map snd |> Array.ofList |> DenseVector.ofArray

    let ff3Betas = ff3X.QR().Solve(ff3Y)
    let ff3Residuals = ff3Y - (ff3X * ff3Betas)
    let ff3Mse = ff3Residuals.PointwisePower(2).Mean()
    { Betas = ff3Betas.ToArray(); Residuals = ff3Residuals.ToArray(); Mse = ff3Mse }

let longCapmModel = fitCapmModel longRd
let longShortCapmModel = fitCapmModel longShortRd

let longff3mModel = fitff3Model longRd
let longShortff3Model = fitff3Model longShortRd


In [18]:
let calculateCapmAlpha (model: RegressionResults) (data: RegData list) =
    let capmAlpha = model.Betas.[0]

    let capmX = data |> List.map (fun r -> [|1.0; r.MktRf|]) |> List.toArray |> DenseMatrix.ofRowArrays

    let residuals = model.Residuals
    let n = float residuals.Length

    let XTX = capmX.TransposeThisAndMultiply(capmX)
    let XTX_inv = XTX.Inverse()

    let sumSquaredResiduals = residuals |> Array.map (fun x -> x * x) |> Array.sum
    let residualsVar = sumSquaredResiduals / (n - 2.0)
    let capmAlphaVar = XTX_inv.[0, 0] * residualsVar
    let capmAlphaStdErr = sqrt capmAlphaVar

    let capmAlphaT = capmAlpha / capmAlphaStdErr

    (capmAlpha, capmAlphaStdErr, capmAlphaT)

let (longCapmAlpha, longCapmAlphaStdErr, longCapmAlphaT) = calculateCapmAlpha longCapmModel longRd
let (longShortCapmAlpha, longShortCapmAlphaStdErr, longShortCapmAlphaT) = calculateCapmAlpha longShortCapmModel longShortRd

let calculateCapmBeta (model: RegressionResults) (data: RegData list) =
    let capmBeta = model.Betas.[1]

    let capmX = data |> List.map (fun r -> [|1.0; r.MktRf|]) |> List.toArray |> DenseMatrix.ofRowArrays

    let residuals = model.Residuals
    let n = float residuals.Length

    let XTX = capmX.TransposeThisAndMultiply(capmX)
    let XTX_inv = XTX.Inverse()

    let sumSquaredResiduals = residuals |> Array.map (fun x -> x * x) |> Array.sum
    let residualsVar = sumSquaredResiduals / (n - 2.0)
    let capmBetaVar = XTX_inv.[1, 1] * residualsVar
    let capmBetaStdErr = sqrt capmBetaVar

    let capmBetaT = capmBeta / capmBetaStdErr

    (capmBeta, capmBetaStdErr, capmBetaT)

let (longCapmBeta, longCapmBetaStdErr, longCapmBetaT) = calculateCapmBeta longCapmModel longRd
let (longShortCapmBeta, longShortCapmBetaStdErr, longShortCapmBetaT) = calculateCapmBeta longShortCapmModel longShortRd

let calculateCapmInformationRatio (model: RegressionResults) =
    let residuals = model.Residuals
    let capmAlpha = model.Betas.[0]

    let trackingError = residuals |> Statistics.StandardDeviation
    let capmIR = capmAlpha / trackingError

    (trackingError, capmIR)

let (longTrackingError, longCapmIR) = calculateCapmInformationRatio longCapmModel
let (longShortTrackingError, longShortCapmIR) = calculateCapmInformationRatio longShortCapmModel


Code needed to get Fama-French 3-factor statistics

In [19]:
let calculateFf3Alpha (model: RegressionResults) (data: RegData list) =
    let ff3Alpha = model.Betas.[0]

    let ff3X = data |> List.map (fun r -> [|1.0; r.MktRf; r.Hml; r.Smb|]) |> List.toArray |> DenseMatrix.ofRowArrays

    let residuals = model.Residuals
    let n = float residuals.Length

    let XTX = ff3X.TransposeThisAndMultiply(ff3X)
    let XTX_inv = XTX.Inverse()

    let sumSquaredResiduals = residuals |> Array.map (fun x -> x * x) |> Array.sum
    let residualsVar = sumSquaredResiduals / (n - 4.0)
    let ff3AlphaVar = XTX_inv.[0, 0] * residualsVar
    let ff3AlphaStdErr = sqrt ff3AlphaVar

    let ff3AlphaT = ff3Alpha / ff3AlphaStdErr

    (ff3Alpha, ff3AlphaStdErr, ff3AlphaT)

let (longFf3Alpha, longFf3AlphaStdErr, longFf3AlphaT) = calculateFf3Alpha longff3mModel longRd
let (longShortFf3Alpha, longShortFf3AlphaStdErr, longShortFf3AlphaT) = calculateFf3Alpha longShortff3Model longShortRd

let calculateFf3Measures (model: RegressionResults) (data: RegData list) =
    let hmlBeta = model.Betas.[2]
    let smbBeta = model.Betas.[3]

    let ff3X = data |> List.map (fun r -> [|1.0; r.MktRf; r.Hml; r.Smb|]) |> List.toArray |> DenseMatrix.ofRowArrays
    let residuals = model.Residuals
    let n = float residuals.Length

    let residualsSquaredSum = residuals |> Array.map (fun x -> x * x) |> Array.sum
    let residualsVar = residualsSquaredSum / (n - 4.0)

    let XTX = ff3X.TransposeThisAndMultiply(ff3X)
    let XTX_inv = XTX.Inverse()

    let hmlBetaVar = XTX_inv.[2, 2] * residualsVar
    let smbBetaVar = XTX_inv.[3, 3] * residualsVar

    let hmlBetaStdErr = sqrt hmlBetaVar
    let smbBetaStdErr = sqrt smbBetaVar

    let hmlBetaT = hmlBeta / hmlBetaStdErr
    let smbBetaT = smbBeta / smbBetaStdErr

    let trackingError = residuals |> Array.map (fun x -> x * x) |> Array.sum |> sqrt
    let ff3Alpha = model.Betas.[0]
    let ff3IR = ff3Alpha / trackingError

    (hmlBeta, hmlBetaStdErr, hmlBetaT, smbBeta, smbBetaStdErr, smbBetaT, trackingError, ff3IR)

let (longHmlBeta, longHmlBetaStdErr, longHmlBetaT, longSmbBeta, longSmbBetaStdErr, longSmbBetaT, longTrackingError, longFf3IR) = calculateFf3Measures longff3mModel longRd

let (longShortHmlBeta, longShortHmlBetaStdErr, longShortHmlBetaT, longShortSmbBeta,
 longShortSmbBetaStdErr, longShortSmbBetaT, longShortTrackingError, longShortFf3IR) = calculateFf3Measures longShortff3Model longShortRd


Calculate measures

In [20]:
// average annualized excess returns
let longavgExcessReturnFirstHalf = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "First Half" long)
let longavgExcessReturnSecondHalf = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "Second Half" long)
let longavgExcessReturnFullPeriod = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "Full Period" long)

let longShortavgExcessReturnFirstHalf = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "First Half" longShort)
let longShortavgExcessReturnSecondHalf = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "Second Half" longShort)
let longShortavgExcessReturnFullPeriod = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "Full Period" longShort)

let vwMktRfavgExcessReturnFirstHalf = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "First Half" vwMktRf)
let vwMktRfavgExcessReturnSecondHalf = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "Second Half" vwMktRf)
let vwMktRfavgExcessReturnFullPeriod = annualizeavgExcessReturn (filterReturnsByTimePeriodsSign "Full Period" vwMktRf)

// annualized Sharpe ratios
let longSharpeFirstHalf = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "First Half" long)
let longSharpeSecondHalf = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "Second Half" long)
let longSharpeFullPeriod = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "Full Period" long)

let longShortSharpeFirstHalf = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "First Half" longShort)
let longShortSharpeSecondHalf = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "Second Half" longShort)
let longShortSharpeFullPeriod = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "Full Period" longShort)

let vwMktRfSharpeFirstHalf = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "First Half" vwMktRf)
let vwMktRfSharpeSecondHalf = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "Second Half" vwMktRf)
let vwMktRfSharpeFullPeriod = annualizeSharpeRatio (filterReturnsByTimePeriodsSign "Full Period" vwMktRf)

// CAPM alphas and t-statistics
let capmModelLongFirstHalf = fitCapmModel (filterReturnsByTimePeriodsRd "First Half" longRd)
let capmModelLongSecondHalf = fitCapmModel (filterReturnsByTimePeriodsRd "Second Half" longRd)
let capmModelLongFullPeriod = fitCapmModel (filterReturnsByTimePeriodsRd "Full Period" longRd)

let capmModelLongShortFirstHalf = fitCapmModel (filterReturnsByTimePeriodsRd "First Half" longShortRd)
let capmModelLongShortSecondHalf = fitCapmModel (filterReturnsByTimePeriodsRd "Second Half" longShortRd)
let capmModelLongShortFullPeriod = fitCapmModel (filterReturnsByTimePeriodsRd "Full Period" longShortRd)

let (longCapmAlphaFirstHalf, _, longCapmAlphaTStatFirstHalf) = calculateCapmAlpha capmModelLongFirstHalf (filterReturnsByTimePeriodsRd "First Half" longRd)
let (longCapmAlphaSecondHalf, _, longCapmAlphaTStatSecondHalf) = calculateCapmAlpha capmModelLongSecondHalf (filterReturnsByTimePeriodsRd "Second Half" longRd)
let (longCapmAlphaFullPeriod, _, longCapmAlphaTStatFullPeriod) = calculateCapmAlpha capmModelLongFullPeriod (filterReturnsByTimePeriodsRd "Full Period" longRd)

let (longShortCapmAlphaFirstHalf, _, longShortCapmAlphaTStatFirstHalf) = calculateCapmAlpha capmModelLongShortFirstHalf (filterReturnsByTimePeriodsRd "First Half" longShortRd)
let (longShortCapmAlphaSecondHalf, _, longShortCapmAlphaTStatSecondHalf) = calculateCapmAlpha capmModelLongShortSecondHalf (filterReturnsByTimePeriodsRd "Second Half" longShortRd)
let (longShortCapmAlphaFullPeriod, _, longShortCapmAlphaTStatFullPeriod) = calculateCapmAlpha capmModelLongShortFullPeriod (filterReturnsByTimePeriodsRd "Full Period" longShortRd)

// Fama-French 3-factor alphas and t-statistics
let ff3ModelLongFirstHalf = fitff3Model (filterReturnsByTimePeriodsRd "First Half" longRd)
let ff3ModelLongSecondHalf = fitff3Model (filterReturnsByTimePeriodsRd "Second Half" longRd)
let ff3ModelLongFullPeriod = fitff3Model (filterReturnsByTimePeriodsRd "Full Period" longRd)

let ff3ModelLongShortFirstHalf = fitff3Model (filterReturnsByTimePeriodsRd "First Half" longShortRd)
let ff3ModelLongShortSecondHalf = fitff3Model (filterReturnsByTimePeriodsRd "Second Half" longShortRd)
let ff3ModelLongShortFullPeriod = fitff3Model (filterReturnsByTimePeriodsRd "Full Period" longShortRd)

let (longFf3AlphaFirstHalf, _, longFf3AlphaTStatFirstHalf) = calculateFf3Alpha ff3ModelLongFirstHalf (filterReturnsByTimePeriodsRd "First Half" longRd)
let (longFf3AlphaSecondHalf, _, longFf3AlphaTStatSecondHalf) = calculateFf3Alpha ff3ModelLongSecondHalf (filterReturnsByTimePeriodsRd "Second Half" longRd)
let (longFf3AlphaFullPeriod, _, longFf3AlphaTStatFullPeriod) = calculateFf3Alpha ff3ModelLongFullPeriod (filterReturnsByTimePeriodsRd "Full Period" longRd)

let (longShortFf3AlphaFirstHalf, _, longShortFf3AlphaTStatFirstHalf) = calculateFf3Alpha ff3ModelLongShortFirstHalf (filterReturnsByTimePeriodsRd "First Half" longShortRd)
let (longShortFf3AlphaSecondHalf, _, longShortFf3AlphaTStatSecondHalf) = calculateFf3Alpha ff3ModelLongShortSecondHalf (filterReturnsByTimePeriodsRd "Second Half" longShortRd)
let (longShortFf3AlphaFullPeriod, _, longShortFf3AlphaTStatFullPeriod) = calculateFf3Alpha ff3ModelLongShortFullPeriod (filterReturnsByTimePeriodsRd "Full Period" longShortRd)

// information ratios
let (_, longCapmIRFirstHalf) = calculateCapmInformationRatio capmModelLongFirstHalf
let (_, longCapmIRSecondHalf) = calculateCapmInformationRatio capmModelLongSecondHalf
let (_, longCapmIRFullPeriod) = calculateCapmInformationRatio capmModelLongFullPeriod

let (_, longShortCapmIRFirstHalf) = calculateCapmInformationRatio capmModelLongShortFirstHalf
let (_, longShortCapmIRSecondHalf) = calculateCapmInformationRatio capmModelLongShortSecondHalf
let (_, longShortCapmIRFullPeriod) = calculateCapmInformationRatio capmModelLongShortFullPeriod

Merge everything together in a table

In [21]:
let performanceTable = [
    {| Portfolio = "Long"; Period = "First Half"; AvgExcessReturn = longavgExcessReturnFirstHalf; SharpeRatio = longSharpeFirstHalf; CapmAlpha = longCapmAlphaFirstHalf; CapmAlphaTStat = longCapmAlphaTStatFirstHalf; FF3Alpha = longFf3AlphaFirstHalf; FF3AlphaTStat = longFf3AlphaTStatFirstHalf; InformationRatio = longCapmIRFirstHalf |};
    {| Portfolio = "Long"; Period = "Second Half"; AvgExcessReturn = longavgExcessReturnSecondHalf; SharpeRatio = longSharpeSecondHalf; CapmAlpha = longCapmAlphaSecondHalf; CapmAlphaTStat = longCapmAlphaTStatSecondHalf; FF3Alpha = longFf3AlphaSecondHalf; FF3AlphaTStat = longFf3AlphaTStatSecondHalf; InformationRatio = longCapmIRSecondHalf |};
    {| Portfolio = "Long"; Period = "Full Period"; AvgExcessReturn = longavgExcessReturnFullPeriod; SharpeRatio = longSharpeFullPeriod; CapmAlpha = longCapmAlphaFullPeriod; CapmAlphaTStat = longCapmAlphaTStatFullPeriod; FF3Alpha = longFf3AlphaFullPeriod; FF3AlphaTStat = longFf3AlphaTStatFullPeriod; InformationRatio = longCapmIRFullPeriod |};

    {| Portfolio = "Long-Short"; Period = "First Half"; AvgExcessReturn = longShortavgExcessReturnFirstHalf; SharpeRatio = longShortSharpeFirstHalf; CapmAlpha = longShortCapmAlphaFirstHalf; CapmAlphaTStat = longShortCapmAlphaTStatFirstHalf; FF3Alpha = longShortFf3AlphaFirstHalf; FF3AlphaTStat = longShortFf3AlphaTStatFirstHalf; InformationRatio = longShortCapmIRFirstHalf |};
    {| Portfolio = "Long-Short"; Period = "Second Half"; AvgExcessReturn = longShortavgExcessReturnSecondHalf; SharpeRatio = longShortSharpeSecondHalf; CapmAlpha = longShortCapmAlphaSecondHalf; CapmAlphaTStat = longShortCapmAlphaTStatSecondHalf; FF3Alpha = longShortFf3AlphaSecondHalf; FF3AlphaTStat = longShortFf3AlphaTStatSecondHalf; InformationRatio = longShortCapmIRSecondHalf |};
    {| Portfolio = "Long-Short"; Period = "Full Period"; AvgExcessReturn = longShortavgExcessReturnFullPeriod; SharpeRatio = longShortSharpeFullPeriod; CapmAlpha = longShortCapmAlphaFullPeriod; CapmAlphaTStat = longShortCapmAlphaTStatFullPeriod; FF3Alpha = longShortFf3AlphaFullPeriod; FF3AlphaTStat = longShortFf3AlphaTStatFullPeriod; InformationRatio = longShortCapmIRFullPeriod |};

    {| Portfolio = "vwMktRf"; Period = "First Half"; AvgExcessReturn = vwMktRfavgExcessReturnFirstHalf; SharpeRatio = vwMktRfSharpeFirstHalf; CapmAlpha = 0.; CapmAlphaTStat = 0.; FF3Alpha = 0.; FF3AlphaTStat = 0.; InformationRatio = 0. |};
    {| Portfolio = "vwMktRf"; Period = "Second Half"; AvgExcessReturn = vwMktRfavgExcessReturnSecondHalf; SharpeRatio = vwMktRfSharpeSecondHalf; CapmAlpha = 0.; CapmAlphaTStat = 0.; FF3Alpha = 0.; FF3AlphaTStat = 0.; InformationRatio = 0. |};
    {| Portfolio = "vwMktRf"; Period = "Full Period"; AvgExcessReturn = vwMktRfavgExcessReturnFullPeriod; SharpeRatio = vwMktRfSharpeFullPeriod; CapmAlpha = 0.; CapmAlphaTStat = 0.; FF3Alpha = 0.; FF3AlphaTStat = 0.; InformationRatio = 0. |};
]

let columnOrder = ["Portfolio"; "Period"; "AvgExcessReturn"; "SharpeRatio"; "CapmAlpha"; "CapmAlphaTStat"; "FF3Alpha"; "FF3AlphaTStat"; "InformationRatio"]
let orderedPerformanceDF = performanceTable |> Frame.ofRecords |> Frame.sliceCols columnOrder

orderedPerformanceDF.Print()

     Portfolio  Period      AvgExcessReturn        SharpeRatio         CapmAlpha              CapmAlphaTStat     FF3Alpha               FF3AlphaTStat        InformationRatio     
0 -> Long       First Half  0,0003359849021016679  0,01986284720377159 0,001369773400013849   1,0455570617417893 0,0023088353160840695  2,113091888741322    0,09353610647769213  
1 -> Long       Second Half 0,013036387381669411   1,0374004530571217  0,00045349049791219387 0,6723214639575283 -0,0002845587254554084 -0,45304076663688386 0,06268639097543324  
2 -> Long       Full Period 0,006686186141885539   0,4462327117508477  0,0004096462410146534  0,5376734976756791 0,0004906280602022736  0,7610451293258454   0,034200039320133525 
3 -> Long-Short First Half  0,0017858663308349594  0,33177620249816925 0,0018093006463858303  1,087227417032686  0,0024207450385800556  1,4329239102355136   0,09726395924830937  
4 -> Long-Short Second Half 0,002493323985919797   0,5446117729990867  0,00398477176839208    2,845188234

### Portfolio Optimization


In [22]:
type StockData =
    { Symbol : string 
      Date : DateTime
      Return : float }

In [23]:
let ff3 = getFF3 Frequency.Monthly |> Array.toList

// Transform to a StockData record type.
let ff3StockData =
    [ 
       ff3 |> List.map(fun x -> {Symbol="HML";Date=x.Date;Return=x.Hml})
       ff3 |> List.map(fun x -> {Symbol="MktRf";Date=x.Date;Return=x.MktRf})
       ff3 |> List.map(fun x -> {Symbol="Smb";Date=x.Date;Return=x.Smb})
    ] |> List.concat

Get ETFs information

In [24]:
let tickers = 
    [ 
        "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

let tickPrices = 
    YahooFinance.History(
        tickers,
        startDate = DateTime(2007,5,1),
        interval = Interval.Monthly)

tickPrices[..3]

index value 0 { Symbol = "VTI"\n Date = 01/05/2007 00:00:00\n Open = 73.65000153\n High = 76.47000122\n Low = 73.22000122\n Close = 76.31999969\n AdjustedClose = 56.36239624\n Volume = 8888000M } Symbol VTI Date 2007-05-01 00:00:00Z Open 73.6500015258789 High 76.47000122070312 Low 73.22000122070312 Close 76.31999969482422 AdjustedClose 56.362396240234375 Volume 8888000 1 { Symbol = "VTI"\n Date = 01/06/2007 00:00:00\n Open = 76.52500153\n High = 76.80999756\n Low = 73.58000183\n Close = 74.69000244\n AdjustedClose = 55.158638\n Volume = 12072000M } Symbol VTI Date 2007-06-01 00:00:00Z Open 76.5250015258789 High 76.80999755859375 Low 73.58000183105469 Close 74.69000244140625 AdjustedClose 55.15863800048828 Volume 12072000 2 { Symbol = "VTI"\n Date = 01/07/2007 00:00:00\n Open = 74.75\n High = 77.35500336\n Low = 71.90499878\n Close = 72.05000305\n AdjustedClose = 53.41179276\n Volume = 12724800M } Symbol VTI Date 2007-07-01 00:00:00Z Open 74.75 High 77.3550033569336 Low 71.90499877929688 Close 72.05000305175781 AdjustedClose 53.41179275512695 Volume 12724800 3 { Symbol = "VTI"\n Date = 01/08/2007 00:00:00\n Open = 72.03500366\n High = 74.44999695\n Low = 67.87000275\n Close = 73.11000061\n AdjustedClose = 54.19757462\n Volume = 29641200M } Symbol VTI Date 2007-08-01 00:00:00Z Open 72.03500366210938 High 74.44999694824219 Low 67.87000274658203 Close 73.11000061035156 AdjustedClose 54.19757461547851 Volume 29641200

Function to calculate returns from Price observations

In [25]:
let pricesToReturns (symbol, adjPrices: list<Quote>) =
    adjPrices
    |> List.sortBy (fun x -> x.Date)
    |> List.pairwise
    |> List.map (fun (day0, day1) ->
        let r = day1.AdjustedClose / day0.AdjustedClose - 1.0 
        { Symbol = symbol 
          Date = day1.Date 
          Return = r })

let tickReturns =
    tickPrices
    |> List.groupBy (fun x -> x.Symbol)
    |> List.collect pricesToReturns

let rf = Map [ for x in ff3 do x.Date, x.Rf ]

let standardInvestmentsExcess =
    let maxff3Date = ff3 |> List.map(fun x -> x.Date) |> List.max
    tickReturns
    |> List.filter(fun x -> x.Date <= maxff3Date)
    |> List.map(fun x -> 
        match Map.tryFind x.Date rf with 
        | None -> failwith $"why isn't there a rf for {x.Date}"
        | Some rf -> { x with Return = x.Return - rf })



In [26]:
standardInvestmentsExcess
|> List.filter(fun x -> x.Symbol = "VTI" && x.Date.Year = 2021)
|> List.map(fun x -> x.Date.Month, round 4 x.Return)
|> List.take 3


index value 0 (1, 0,0006) Item1 1 Item2 0.0006 1 (2, 0,0314) Item1 2 Item2 0.0314 2 (3, 0,033) Item1 3 Item2 0.033

In [27]:
let returnMap =
    standardInvestmentsExcess
    |> List.map (fun x -> (x.Symbol, x.Date), x.Return)
    |> Map 

Long and Long-Short 

In [28]:
// Function to convert portfolio observations to stock data
let convertToStockData (portfolioObs: SignalPortfolioObs) : StockData =
    { Symbol = portfolioObs.Name
      Date = portfolioObs.Month
      Return = portfolioObs.ExcessReturn }

let longStockData = long |> List.map convertToStockData
let longShortStockData = longShort |> List.map convertToStockData

// Creates a new list that concatenates 'standardInvestmentsExcess', 'longStockData', and 'longShortStockData'
let extendedStandardInvestmentsExcess = 
    standardInvestmentsExcess @ longStockData @ longShortStockData   


In [29]:
// Group and sort stock data by date
let stockDataByDate =
    extendedStandardInvestmentsExcess
    |> List.groupBy(fun x -> x.Date)
    |> List.sortBy (fun (dt, xs) -> dt)

// Find the first month where the number of stocks equals 4
let allAssetsStart =
    stockDataByDate
    |> List.find(fun (month, stocks) -> stocks.Length = 4)
    |> fst

// Find the last month where the number of stocks equals 4
let allAssetsEnd =
    stockDataByDate
    |> List.findBack(fun (month, stocks) -> stocks.Length = 4)
    |> fst

// Filter stock data to include only complete date range
let stockDataByDateComplete =
    extendedStandardInvestmentsExcess
    |> List.filter(fun x -> 
        x.Date >= allAssetsStart &&
        x.Date <= allAssetsEnd)

Covariances

In [30]:
// Create a map of (Symbol, Date) to Return
let returnMap =
    stockDataByDateComplete
    |> List.map (fun x -> (x.Symbol, x.Date), x.Return)
    |> Map

// Define a function to calculate covariance of returns for given symbols
let getCov xId yId =
    // Filter stock data for specified symbol xId
    let xs = 
        stockDataByDateComplete
        |> List.filter (fun x -> x.Symbol = xId)
    // Calculate covariance of return pairs
    [ for x in xs do
        let yLookup = yId, x.Date
        if returnMap.ContainsKey yLookup then
            x.Return, returnMap[yLookup]]
    |> covOfPairs


Mean variance weights

In [31]:
// Function to calculate minimum variance efficient weights for a list of tickers
let getMveWeights (tickers:List<string>) =
    // Calculate covariance matrix for tickers
    let covariances =
        [ for rowTick in tickers do
            [ for colTick in tickers do
                getCov rowTick colTick ]]
        |> dsharp.tensor

    // Calculate mean returns for each ticker
    let meansByTick =
        extendedStandardInvestmentsExcess
        |> List.groupBy (fun x -> x.Symbol)
        |> List.map (fun (sym, xs) ->
            let symAvg = xs |> List.averageBy (fun x -> x.Return)
            sym, symAvg)
        |> Map

    // Create tensor for mean returns
    let means = 
        [ for ticker in tickers do meansByTick[ticker] ]
        |> dsharp.tensor

    // Solve for weights
    let w' = dsharp.solve(covariances,means)
    let w = w' / w'.sum()   // Normalize weights
    let weights =
        Seq.zip tickers (w.toArray1D<float>())   // Zip tickers and weights
        |> Map.ofSeq    // Convert to map

    weights


In [32]:
let portfolioMonthReturn weights monthData =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum

let weightsLongPlusAssets = getMveWeights ["Long";"VTI";"BND"]
weightsLongPlusAssets

key,value
BND,0.8237399458885193
Long,-0.3082203269004822
VTI,0.4844803214073181


In [33]:
let stockDataGroupedByDate = 
    stockDataByDateComplete
    |> List.groupBy (fun x -> x.Date)

let portMveLong =
    stockDataGroupedByDate
    |> List.map(fun (date, data) -> 
        { Symbol = "MVE"
          Date = date
          Return = portfolioMonthReturn weightsLongPlusAssets data })

let weightsLongShortPlusAssets = getMveWeights ["Long-short";"VTI";"BND"]
weightsLongShortPlusAssets

key,value
BND,0.5004767179489136
Long-short,0.36945128440856934
VTI,0.13007204234600067


In [34]:
type Return = 
    { 
        Month: DateTime
        Name: string
        Return: float 
    }

In [35]:
// Function to calculate minimum variance efficient weights for a list of tickers
let getMveWeights (tickers:List<string>) (weights:List<float>) =
    // Calculate covariance matrix for tickers
    let covariances =
        [ for rowTick in tickers do
            [ for colTick in tickers do
                getCov rowTick colTick ]]
        |> dsharp.tensor

    // Calculate mean returns for each ticker
    let meansByTick =
        extendedStandardInvestmentsExcess
        |> List.groupBy (fun x -> x.Symbol)
        |> List.map (fun (sym, xs) ->
            let symAvg = xs |> List.averageBy (fun x -> x.Return)
            sym, symAvg)
        |> Map

    // Create tensor for mean returns
    let means = 
        [ for ticker in tickers do meansByTick[ticker] ]
        |> dsharp.tensor

    // Solve for weights
    let w' = dsharp.solve(covariances,means)
    let w = w' / w'.sum()  // Normalize weights
    let weights =
        Seq.zip tickers (Array.ofList weights)  // Zip tickers and weights
        |> Map.ofSeq   // Convert to map

    weights

// Function to calculate the portfolio return for a given month
let portfolioMonthReturn weights monthData =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum

// Get minimum variance efficient weights for "VTI" and "BND"
let weightsVTI_BND = getMveWeights ["VTI";"BND"] [0.6; 0.4]

// Group stock data by date
let stockDataGroupedByDate = 
    stockDataByDateComplete
    |> List.groupBy (fun x -> x.Date)
    
// Calculate the portfolio return for "VTI_BND"
let portVTI_BND =
    stockDataGroupedByDate
    |> List.map(fun (date, data) -> 
        { Symbol = "VTI_BND"
          Date = date
          Return = portfolioMonthReturn weightsVTI_BND data })

// Display the first three elements of 'portVTI_BND'
portVTI_BND[..2]


index value 0 { Symbol = "VTI_BND"\n Date = 01/06/2007 00:00:00\n Return = -0.01876622847 } Symbol VTI_BND Date 2007-06-01 00:00:00Z Return -0.0187662284715532 1 { Symbol = "VTI_BND"\n Date = 01/07/2007 00:00:00\n Return = -0.01918462112 } Symbol VTI_BND Date 2007-07-01 00:00:00Z Return -0.019184621123252563 2 { Symbol = "VTI_BND"\n Date = 01/08/2007 00:00:00\n Return = 0.01018100952 } Symbol VTI_BND Date 2007-08-01 00:00:00Z Return 0.010181009515235734

In [36]:
// Calculate the monthly returns for the mean-variance efficient long-short portfolio with VTI and BND
let portMveLongShort =
    stockDataGroupedByDate
    |> List.map(fun (date, data) -> 
        { Symbol = "Change in COWC Long-Short"
          Date = date
          Return = portfolioMonthReturn weightsLongShortPlusAssets data })

In [37]:
// Function to create a cumulative return chart for a list of stock returns
let makeCumulativeChartdf (returns:List<StockData>) =
    // Get the first observation in the returns list
    let firstObs = returns[0]
    returns
    // Map each return to a (Date, Return) pair
    |> List.map (fun x ->x.Date, x.Return)
    // Calculate the cumulative return
    |> cumulativeReturn
    // Generate the line chart
    |> Chart.Line
    // Add trace info to the chart
    |> Chart.withTraceInfo(Name = $"{firstObs.Symbol}")

// Function to apply leverage to a list of stock returns
let applyLeveragedf (xs:List<StockData> ) =
    // Calculate the annualized volatility
    let annualizedVol = sqrt(12.0) * (xs |> Seq.stDevBy(fun x -> x.Return))
    let targetVol = 0.1
    xs 
    // Adjust each return by the ratio of target volatility to annualized volatility
    |> List.map(fun x -> 
        { x with Return = (targetVol/annualizedVol) * x.Return })    

// Function to create a cumulative return chart for a list of leveraged stock returns
let makeCumulativeChartLeverageddf (returns:List<StockData>) =
    // Get the first observation in the returns list
    let firstObs = returns[0]
    returns
    // Apply leverage to the returns
    |> applyLeveragedf
    // Map each return to a (Date, Return) pair
    |> List.map (fun x -> x.Date, x.Return)
    // Calculate the cumulative return
    |> cumulativeReturn
    // Generate the line chart
    |> Chart.Line
    // Add trace info to the chart
    |> Chart.withTraceInfo(Name = $"{firstObs.Symbol}")


Let's now try with different weights

In [38]:
// Get minimum variance efficient weights for "VTI" and "BND"
let weightsVTI_BND_Inv = getMveWeights ["VTI";"BND"] [0.4; 0.6]
    
// Calculate the portfolio return for "VTI_BND"
let portVTI_BND_Inv =
    stockDataGroupedByDate
    |> List.map(fun (date, data) -> 
        { Symbol = "VTI_BND"
          Date = date
          Return = portfolioMonthReturn weightsVTI_BND_Inv data })

In [39]:
let p6040 =
    portVTI_BND
    |> makeCumulativeChartdf
    |> Chart.withXAxisStyle "Year"
    |> Chart.withYAxisStyle "Cumulative Return"
    |> Chart.withTraceInfo(Name = $"60 Stock / 40 Bond")

let p4060 =
    portVTI_BND_Inv
    |> makeCumulativeChartdf
    |> Chart.withXAxisStyle "Year"
    |> Chart.withYAxisStyle "Cumulative Return"
    |> Chart.withTraceInfo(Name = $"40 Stock / 60 Bond")
    
let longCum =
    portMveLong
    |> makeCumulativeChartdf
    |> Chart.withXAxisStyle "Year"
    |> Chart.withYAxisStyle "Cumulative Return"
    |> Chart.withTraceInfo(Name = $"Long + Assets")  

let longShortCum =
    portMveLongShort
    |> makeCumulativeChartdf
    |> Chart.withXAxisStyle "Year"
    |> Chart.withYAxisStyle "Cumulative Return"
    |> Chart.withTraceInfo(Name = $"Long-Short + Assets")
    
let combinedChart =
    [p6040; p4060; longCum; longShortCum]
    |> Chart.combine
    
combinedChart

<!-- Plotly chart will be drawn inside this DIV -->

In [40]:
let p6040Vol =
    portVTI_BND
    |> makeCumulativeChartLeverageddf
    |> Chart.withXAxisStyle "Year"
    |> Chart.withYAxisStyle "Cumulative Return"
    |> Chart.withTraceInfo(Name = $"60 Stock / 40 Bond")

let p4060Vol =
    portVTI_BND_Inv
    |> makeCumulativeChartdf
    |> Chart.withXAxisStyle "Year"
    |> Chart.withYAxisStyle "Cumulative Return"
    |> Chart.withTraceInfo(Name = $"40 Stock / 60 Bond")

let longCumVol =
    portMveLong
    |> makeCumulativeChartLeverageddf
    |> Chart.withXAxisStyle "Year"
    |> Chart.withYAxisStyle "Cumulative Return"
    |> Chart.withTraceInfo(Name = $"Long + Assets")

let longShortCumVol =
    portMveLongShort
    |> makeCumulativeChartLeverageddf
    |> Chart.withXAxisStyle "Year"
    |> Chart.withYAxisStyle "Cumulative Return"
    |> Chart.withTraceInfo(Name = $"Long-Short + Assets")

let combinedChart =
    [p6040Vol; p4060Vol; longCumVol; longShortCumVol]
    |> Chart.combine
    
combinedChart

<!-- Plotly chart will be drawn inside this DIV -->

In [41]:
// Function to calculate annualized performance metrics
let calculateAnnualizedMetrics (portfolio: StockData list) =
    let returns = portfolio |> List.map(fun obs -> obs.Return)
    let avgRets = returns |> List.average
    let stdevRet = returns |> stDev

    let annualizedAvgExcessReturns = 12.0 * avgRets
    let annualizedSharpeRatio = sqrt(12.0) * (avgRets/stdevRet)

    [annualizedAvgExcessReturns; annualizedSharpeRatio]

In [42]:
let performanceTable = [
    {| Portfolio = "Long + Assets"; AvgExcessReturn = [calculateAnnualizedMetrics portMveLong][0][0]; SharpeRatio = [calculateAnnualizedMetrics portMveLong][0][1]|};
    {| Portfolio = "Long Short + Assets"; AvgExcessReturn = [calculateAnnualizedMetrics portMveLongShort][0][0]; SharpeRatio = [calculateAnnualizedMetrics portMveLongShort][0][1]|};
    {| Portfolio = "60 Stock / 40 Bond"; AvgExcessReturn = [calculateAnnualizedMetrics portVTI_BND][0][0]; SharpeRatio = [calculateAnnualizedMetrics portVTI_BND][0][1]|};
    {| Portfolio = "40 Stock / 60 Bond"; AvgExcessReturn = [calculateAnnualizedMetrics portVTI_BND_Inv][0][0]; SharpeRatio = [calculateAnnualizedMetrics portVTI_BND_Inv][0][1]|};
]

let columnOrder = ["Portfolio"; "AvgExcessReturn"; "SharpeRatio"]
let orderedPerformanceDF = performanceTable |> Frame.ofRecords |> Frame.sliceCols columnOrder

orderedPerformanceDF.Print()

     Portfolio           AvgExcessReturn      SharpeRatio        
0 -> Long + Assets       0,04022060765244497  0,8867118225787891 
1 -> Long Short + Assets 0,04556854757829241  1,4330035939668062 
2 -> 60 Stock / 40 Bond  0,07160604612010416  0,7075959644381786 
3 -> 40 Stock / 60 Bond  0,059995030213363436 0,8423559169007127 

